In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1f_9EbnywCj35EBUA32sueigxBjBJwALr'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('War and Peace by Leo Tolstoy (ru).txt')

In [ ]:
import pandas as pd

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().set("spark.ui.port", "4050")

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

*`ngrok может отработать не с первого раза, повторите при необходимости.`*

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-05-29 18:38:02--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.144.219.72, 35.169.70.242, 34.202.43.88, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.144.219.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  21.6MB/s    in 0.6s    

2021-05-29 18:38:03 (21.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


Подсчитай кол-во слов в документе "War and Peace by Leo Tolstoy (ru).txt"

In [ ]:
# код
import chardet    
df = open('./War and Peace by Leo Tolstoy (ru).txt', 'rb').read()
result = chardet.detect(df)
charenc = result['encoding']

In [ ]:
charenc

'windows-1251'

In [ ]:
war_and_peace = spark.read.csv('./War and Peace by Leo Tolstoy (ru).txt', sep='\n', encoding=charenc)

In [ ]:
war_and_peace.show()

+--------------------+
|                 _c0|
+--------------------+
|Война и мир. Книга 1|
|Лев Николаевич То...|
|      Война и мир #1|
|В книгу вошли пер...|
|          Том первый|
|        Часть первая|
|                   I|
|– Eh bien, mon pr...|
|Так говорила в ию...|
|«Si vous n’avez r...|
|– Dieu, quelle vi...|
|Он говорил на том...|
|– Avant tout dite...|
|– Как можно быть ...|
|– А праздник англ...|
|– Я думала, что н...|
|– Ежели бы знали,...|
|– Ne me tourmente...|
|– Как вам сказать...|
|Князь Василий гов...|
+--------------------+
only showing top 20 rows



In [ ]:
pattern_punct = '[!@"“’«»#$%&\'()*+.,—/:;<=>?^_`{|}~\[\]\d]'
war_and_peace = war_and_peace.withColumn('cleaned', regexp_replace('_c0', pattern_punct, ''))

In [ ]:
war_and_peace.show()

+--------------------+--------------------+
|                 _c0|             cleaned|
+--------------------+--------------------+
|Война и мир. Книга 1|  Война и мир Книга |
|Лев Николаевич То...|Лев Николаевич То...|
|      Война и мир #1|        Война и мир |
|В книгу вошли пер...|В книгу вошли пер...|
|          Том первый|          Том первый|
|        Часть первая|        Часть первая|
|                   I|                   I|
|– Eh bien, mon pr...|– Eh bien mon pri...|
|Так говорила в ию...|Так говорила в ию...|
|«Si vous n’avez r...|Si vous navez rie...|
|– Dieu, quelle vi...|– Dieu quelle vir...|
|Он говорил на том...|Он говорил на том...|
|– Avant tout dite...|– Avant tout dite...|
|– Как можно быть ...|– Как можно быть ...|
|– А праздник англ...|– А праздник англ...|
|– Я думала, что н...|– Я думала что ны...|
|– Ежели бы знали,...|– Ежели бы знали ...|
|– Ne me tourmente...|– Ne me tourmente...|
|– Как вам сказать...|– Как вам сказать...|
|Князь Василий гов...|Князь Васи

In [ ]:
from pyspark.ml.feature import RegexTokenizer
regexTokenizer = RegexTokenizer(inputCol='cleaned', outputCol='tokens', pattern=r'\s+')
war_and_peace = regexTokenizer.transform(war_and_peace)

In [ ]:
war_and_peace.show()

+--------------------+--------------------+--------------------+
|                 _c0|             cleaned|              tokens|
+--------------------+--------------------+--------------------+
|Война и мир. Книга 1|  Война и мир Книга |[война, и, мир, к...|
|Лев Николаевич То...|Лев Николаевич То...|[лев, николаевич,...|
|      Война и мир #1|        Война и мир |     [война, и, мир]|
|В книгу вошли пер...|В книгу вошли пер...|[в, книгу, вошли,...|
|          Том первый|          Том первый|       [том, первый]|
|        Часть первая|        Часть первая|     [часть, первая]|
|                   I|                   I|                 [i]|
|– Eh bien, mon pr...|– Eh bien mon pri...|[– eh, bien, mon,...|
|Так говорила в ию...|Так говорила в ию...|[так, говорила, в...|
|«Si vous n’avez r...|Si vous navez rie...|[si, vous, navez,...|
|– Dieu, quelle vi...|– Dieu quelle vir...|[– dieu, quelle, ...|
|Он говорил на том...|Он говорил на том...|[он, говорил, на,...|
|– Avant tout dite...|– A

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 8.2MB 4.6MB/s 


In [ ]:
import pymorphy2
from nltk.corpus import stopwords
morph = pymorphy2.MorphAnalyzer()
ru_stopwords = stopwords.words('russian')
digits = [str(i) for i in range(10)]
def preprocess(tokens):
    return [morph.normal_forms(word)[0]
            for word in tokens
                if (word[0] not in digits and
                    word not in ru_stopwords)]

In [ ]:
from pyspark.sql.types import ArrayType, StringType
preprocess_udf = udf(preprocess, ArrayType(StringType()))
war_and_peace = war_and_peace.withColumn('finished', preprocess_udf('tokens'))

In [ ]:
war_and_peace.show()

+--------------------+--------------------+--------------------+--------------------+
|                 _c0|             cleaned|              tokens|            finished|
+--------------------+--------------------+--------------------+--------------------+
|Война и мир. Книга 1|  Война и мир Книга |[война, и, мир, к...| [война, мир, книга]|
|Лев Николаевич То...|Лев Николаевич То...|[лев, николаевич,...|[лев, николаевич,...|
|      Война и мир #1|        Война и мир |     [война, и, мир]|        [война, мир]|
|В книгу вошли пер...|В книгу вошли пер...|[в, книгу, вошли,...|[книга, войти, пе...|
|          Том первый|          Том первый|       [том, первый]|            [первый]|
|        Часть первая|        Часть первая|     [часть, первая]|     [часть, первый]|
|                   I|                   I|                 [i]|                 [i]|
|– Eh bien, mon pr...|– Eh bien mon pri...|[– eh, bien, mon,...|[– eh, bien, mon,...|
|Так говорила в ию...|Так говорила в ию...|[так, говор

In [ ]:
number_of_words = war_and_peace.select(size(war_and_peace.finished)\
                               .name('number_of_words'))\
                               .agg(sum('number_of_words')\
                               .name('number_of_words'))

In [ ]:
number_of_words.show()

+---------------+
|number_of_words|
+---------------+
|         299659|
+---------------+



Необходимо обучить модель используя Spark MLlib (модель на ваш выбор, например Decision Tree) и получить accuracy.
Подробнее тут: https://spark.apache.org/docs/latest/ml-classification-regression.html

In [ ]:
id='13yfAoONwq4rS5XrTv3IrcqcFcdgfvK9V'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-train.txt')

id='1VE_9x0LQvOJpHXbXp_RMPl3Q4wRUuOok'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-test.txt')

In [ ]:
training = spark.read.format("libsvm") \
  .option("numFeatures", "784") \
  .load("./mnist-digits-train.txt")
test = spark.read.format("libsvm") \
  .option("numFeatures", "784") \
  .load("./mnist-digits-test.txt")

In [ ]:
training.show(5)
test.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  5.0|(784,[152,153,154...|
|  0.0|(784,[127,128,129...|
|  4.0|(784,[160,161,162...|
|  1.0|(784,[158,159,160...|
|  9.0|(784,[208,209,210...|
+-----+--------------------+
only showing top 5 rows

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  7.0|(784,[202,203,204...|
|  2.0|(784,[94,95,96,97...|
|  1.0|(784,[128,129,130...|
|  0.0|(784,[124,125,126...|
|  4.0|(784,[150,151,159...|
+-----+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
rf = RandomForestClassifier(labelCol='label', 
                            featuresCol='features', 
                            numTrees=500)
model = rf.fit(training)

In [ ]:
predictions = model.transform(test)
predictions.select('label', 'features').show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  7.0|(784,[202,203,204...|
|  2.0|(784,[94,95,96,97...|
|  1.0|(784,[128,129,130...|
|  0.0|(784,[124,125,126...|
|  4.0|(784,[150,151,159...|
+-----+--------------------+
only showing top 5 rows



In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='label', 
                                              predictionCol='prediction', 
                                              metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print(f'accuracy: {accuracy}')

accuracy: 0.8631


Тут лежат данные для решения задач на sql. Необходимо написать код который формирует из данных датафреймы и используя sqlContext написать sql код с решением. Приветствуется наличие нескольких вариантов решения задачи.

In [ ]:
id='1kUIrskM0zNH8u71G9M1BkHjRQYxvgAvh'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('data.zip')
!unzip data.zip

Archive:  data.zip
replace data/regions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/regions.csv        
replace data/departments.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/departments.csv    
replace data/jobs.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/jobs.csv           
replace data/locations.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/locations.csv      
replace data/country.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/country.csv        
replace data/employees.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/employees.csv      
replace data/job_history.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/job_history.csv    


In [ ]:
regions = spark.read.csv('./data/regions.csv', header=True, sep='\t', inferSchema=True)
regions.createOrReplaceTempView('regions')

departments = spark.read.csv('./data/departments.csv', header=True, sep='\t', inferSchema=True)
departments.createOrReplaceTempView('departments')

jobs = spark.read.csv('./data/jobs.csv', header=True, sep='\t', inferSchema=True)
jobs.createOrReplaceTempView('jobs')

locations = spark.read.csv('./data/locations.csv', header=True, sep='\t', inferSchema=True)
locations.createOrReplaceTempView('locations')

country = spark.read.csv('./data/country.csv', header=True, sep='\t', inferSchema=True)
country.createOrReplaceTempView('country')

employees = spark.read.csv('./data/employees.csv', header=True, sep='\t', inferSchema=True)
employees.createOrReplaceTempView('employees')

job_history = spark.read.csv('./data/job_history.csv', header=True, sep='\t', inferSchema=True)
job_history.createOrReplaceTempView('job_history')

In [ ]:
# посмотрим, что находится в таблицах
regions.show(5)
departments.show(5)
jobs.show(5)
locations.show(5)
country.show(5)
employees.show(5)
job_history.show(5)

+---------+--------------------+
|REGION_ID|         REGION_NAME|
+---------+--------------------+
|        1|              Europe|
|        2|            Americas|
|        3|                Asia|
|        4|Middle East and A...|
+---------+--------------------+

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
|           20|      Marketing|       201|       1800|
|           30|     Purchasing|       114|       1700|
|           40|Human Resources|       203|       2400|
|           50|       Shipping|       121|       1500|
+-------------+---------------+----------+-----------+
only showing top 5 rows

+----------+--------------------+----------+----------+
|    JOB_ID|           JOB_TITLE|MIN_SALARY|MAX_SALARY|
+----------+--------------------+----------+----------+
|   AD_PRES|           President|     20080|   

Кто получает больше всего? Кто меньше всего?

In [ ]:
# код
# получает больше всего
spark.sql(
    """
    select first_name, last_name, salary
    from employees
    where salary = (select max(salary) 
                    from employees) 
    """).show()

+----------+---------+------+
|first_name|last_name|salary|
+----------+---------+------+
|    Steven|     King| 24000|
+----------+---------+------+



In [ ]:
# код
# получает меньше всего
spark.sql(
    """
    select first_name, last_name, salary
    from employees
    where salary = (select min(salary) 
                    from employees) 
    """).show()

+----------+---------+------+
|first_name|last_name|salary|
+----------+---------+------+
|        TJ|    Olson|  2100|
+----------+---------+------+



Выведете топ 5 по зарплате.

In [ ]:
# код
spark.sql(
    """
    select first_name, last_name, salary
    from employees
    order by salary desc
    limit 5 
    """).show()

+----------+---------+------+
|first_name|last_name|salary|
+----------+---------+------+
|    Steven|     King| 24000|
|     Neena|  Kochhar| 17000|
|       Lex|  De Haan| 17000|
|      John|  Russell| 14000|
|     Karen| Partners| 13500|
+----------+---------+------+



Сколько всего регионов? Сколько работников в каждом регионе?

In [ ]:
# код
# сколько всего регионов
spark.sql(
    """
    select count(*) as number_of_regions
    from regions
    """).show()

+-----------------+
|number_of_regions|
+-----------------+
|                4|
+-----------------+



In [ ]:
# какие именно это регионы
spark.sql(
    """
    select region_id, region_name
    from regions
    """).show()

+---------+--------------------+
|region_id|         region_name|
+---------+--------------------+
|        1|              Europe|
|        2|            Americas|
|        3|                Asia|
|        4|Middle East and A...|
+---------+--------------------+



In [ ]:
# сколько работников в каждом регионе
spark.sql(
    """
    select r.region_id, count(e.employee_id) as number_of_employees
    from employees e
    join departments d 
      on e.department_id = d.department_id
    join locations l 
      on d.location_id = l.location_id
    join country c 
      on l.country_id = c.country_id
    join regions r 
      on c.region_id = r.region_id
    group by r.region_id
    """).show()

+---------+-------------------+
|region_id|number_of_employees|
+---------+-------------------+
|        1|                 36|
|        2|                 70|
+---------+-------------------+



Выведете всех работников из Китая.

In [ ]:
# код
spark.sql(
    """
    select first_name, last_name
    from employees e
    join departments d 
      on e.department_id = d.department_id
    join locations l 
      on d.location_id = l.location_id
    join country c 
      on l.country_id = c.country_id
    where country_name = 'China'
    """).show()

+----------+---------+
|first_name|last_name|
+----------+---------+
+----------+---------+



Укажите самую высокооплачиваемою должность.

In [ ]:
# код
spark.sql(
    """
    select job_title, max_salary
    from jobs
    where max_salary = (select max(max_salary) 
                        from jobs) 
    """).show()

+---------+----------+
|job_title|max_salary|
+---------+----------+
|President|     40000|
+---------+----------+



Выведете всех работников связанных с ИТ. Выведете их менеджеров. 

In [ ]:
# код
# вывести всех работников, связанных с ИТ
spark.sql(
    """
    select first_name, last_name
    from employees e
    join departments d 
      on e.department_id = d.department_id
    where department_name like '%IT%'
    """).show()

+----------+---------+
|first_name|last_name|
+----------+---------+
| Alexander|   Hunold|
|     Bruce|    Ernst|
|     David|   Austin|
|     Valli|Pataballa|
|     Diana|  Lorentz|
+----------+---------+



In [ ]:
# вывести менеджеров работников, связанных с ИТ (вариант 1)
# если нужна информация как о менеджерах, так и о работниках из ИТ
spark.sql(
    """
    with managers_and_employees as (
      select 
        m.employee_id as manager_id, 
        m.first_name as manager_first_name,
        m.last_name as manager_last_name,
        e.employee_id as employee_id, 
        e.first_name as employee_first_name, 
        e.last_name as employee_last_name, 
        e.department_id
      from employees e
      left join employees m
      where  m.employee_id = e.manager_id
    )

    select 
      me.manager_id,
      me.manager_first_name,
      me.manager_last_name,
      me.employee_id,
      me.employee_first_name,
      me.employee_last_name,
      d.department_name as employee_department_name
    from managers_and_employees me
    join departments d 
      on me.department_id = d.department_id
    where department_name like '%IT%'
    """).show()

+----------+------------------+-----------------+-----------+-------------------+------------------+------------------------+
|manager_id|manager_first_name|manager_last_name|employee_id|employee_first_name|employee_last_name|employee_department_name|
+----------+------------------+-----------------+-----------+-------------------+------------------+------------------------+
|       102|               Lex|          De Haan|        103|          Alexander|            Hunold|                      IT|
|       103|         Alexander|           Hunold|        104|              Bruce|             Ernst|                      IT|
|       103|         Alexander|           Hunold|        105|              David|            Austin|                      IT|
|       103|         Alexander|           Hunold|        106|              Valli|         Pataballa|                      IT|
|       103|         Alexander|           Hunold|        107|              Diana|           Lorentz|                  

In [ ]:
# вывести менеджеров работников, связанных с ИТ (вариант 2)
# если нужна информация лишь о личности менеджеров
spark.sql(
    """
    with managers_and_employees as (
      select 
        m.employee_id as manager_id, 
        m.first_name as manager_first_name,
        m.last_name as manager_last_name,
        e.employee_id as employee_id, 
        e.first_name as employee_first_name, 
        e.last_name as employee_last_name, 
        e.department_id
      from employees e
      left join employees m
      where  m.employee_id = e.manager_id
    )

    select 
      distinct me.manager_id,
      me.manager_first_name,
      me.manager_last_name
    from managers_and_employees me
    join departments d 
      on me.department_id = d.department_id
    where department_name like '%IT%'
    """).show()

+----------+------------------+-----------------+
|manager_id|manager_first_name|manager_last_name|
+----------+------------------+-----------------+
|       103|         Alexander|           Hunold|
|       102|               Lex|          De Haan|
+----------+------------------+-----------------+



Выведете имя и фамилию работника, его текущую и предыдущую должности и сколько полных недель и дней прошло с момент изменения. 



In [ ]:
# код 
# не совсем было понятно, что считать разницей, в итоге получилось так:
spark.sql(
    """
    with cte as (
      select 
        employee_id,
        job_id,
        to_date(end_date, 'dd.mm.yy') as end_date_,
        row_number() over(partition by employee_id order by to_date(end_date, 'dd.mm.yy') desc) as rank
      from job_history
    )

    select 
      --e.employee_id as employee_id,
      e.first_name as first_name,
      e.last_name as last_name,
      e.job_id as current_job_id,
      cte.job_id as previous_job_id,
      --cte.end_date_ as end_date,
      --to_date(e.hire_date, 'dd.mm.yy') as hire_date,
      datediff(cte.end_date_, to_date(e.hire_date, 'dd.mm.yy')) as days,
      ceil(datediff(cte.end_date_, to_date(e.hire_date, 'dd.mm.yy')) / 7) as weeks
    from employees e
    join cte
      on e.employee_id = cte.employee_id 
      and cte.rank = 1
    """).show()

+----------+---------+--------------+---------------+----+-----+
|first_name|last_name|current_job_id|previous_job_id|days|weeks|
+----------+---------+--------------+---------------+----+-----+
|     Neena|  Kochhar|         AD_VP|         AC_MGR|  -6|    0|
|       Lex|  De Haan|         AD_VP|        IT_PROG|1837|  263|
|       Den| Raphaely|        PU_MAN|       ST_CLERK|1850|  265|
|     Payam| Kaufling|        ST_MAN|       ST_CLERK|1491|  213|
|  Jonathon|   Taylor|        SA_REP|         SA_MAN| 372|   54|
|  Jennifer|   Whalen|       AD_ASST|     AC_ACCOUNT|1110|  159|
|   Michael|Hartstein|        MK_MAN|         MK_REP|1098|  157|
+----------+---------+--------------+---------------+----+-----+



Выведете уникальные телефонные номера

In [ ]:
# код
spark.sql(
    """
    select distinct phone_number
    from employees
    """).show()

+------------------+
|      phone_number|
+------------------+
|011.44.1344.429018|
|      515.127.4566|
|      515.127.4564|
|011.44.1344.429278|
|      515.123.4569|
|      650.124.1434|
|      650.123.2234|
|011.44.1344.498718|
|      650.127.1634|
|      515.127.4561|
|011.44.1345.629268|
|      515.127.4562|
|011.44.1644.429264|
|011.44.1644.429262|
|      650.501.1876|
|      650.127.1834|
|011.44.1343.529268|
|011.44.1644.429265|
|      515.123.8181|
|      650.507.9833|
+------------------+
only showing top 20 rows



Есть ли сотрудники с одинаковыми фамилиями и сколько их.

In [ ]:
# код
# есть ли сотрудники с одинаковыми фамилиями
spark.sql(
    """
    select last_name, count(*) as number_of_last_names
    from employees
    group by last_name
    having number_of_last_names > 1
    """).show()

+---------+--------------------+
|last_name|number_of_last_names|
+---------+--------------------+
|    Smith|                   2|
|     King|                   2|
|Cambrault|                   2|
|   Taylor|                   2|
+---------+--------------------+



In [ ]:
# общее число сотрудников с одинаковыми фамилиями
spark.sql(
    """
    with cte as (
      select count(*) as number_of_last_names
      from employees
      group by last_name
      having number_of_last_names > 1
    )

    select sum(*) as number_of_people_with_similar_last_names from cte
    """).show()

+----------------------------------------+
|number_of_people_with_similar_last_names|
+----------------------------------------+
|                                       8|
+----------------------------------------+

